## Model Comparison
In this notebook we compared many different models for predicting personality traits. We also built voting ensembles of SVM, RF, MLP and GradientBoostingClassifier. The best result was given by random forest. We also tested SMOTE, due to strong presence of dominant labels in training set. But the result didn't show improvement, since the label distribution in validation set behaves differently.

### Train and validation sets:

The training and validation sets used in this notebook are BERT embeddings and LIWC features concatenated in csv format.

train: comb_train_liwc_embed.csv
val_df comb_val_liwc_embed.csv

Both CSV files can be found in the folder "processes_data/" in subfolders "train/" and "validation/".

### SVM

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns] 

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

reports = []

for trait in traits:
    print(f"\n==== Trait: {trait.capitalize()} ====")

    X_train = train_df[feature_cols].values
    y_train = train_df[trait].values

    X_test = val_df[feature_cols].values
    y_test = val_df[trait].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    svm = SVC(kernel="linear", C=1.0, random_state=42)
    svm.fit(X_train_scaled, y_train)

    y_pred = svm.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print("Accuracy:", acc)
    print("F1-score (macro):", f1)

    reports.append((trait, acc, f1, report))

report_path = "reports/svm.txt"
with open(report_path, "w") as f:
    for trait, acc, f1, rep in reports:
        f.write(f"=== Trait: {trait.capitalize()} ===\n")
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"F1-score (macro): {f1:.4f}\n")
        f.write(rep + "\n\n")


==== Trait: Openness ====
Accuracy: 0.25
F1-score (macro): 0.22873900293255134

==== Trait: Conscientiousness ====
Accuracy: 0.25
F1-score (macro): 0.25250544662309365

==== Trait: Extraversion ====
Accuracy: 0.40625
F1-score (macro): 0.2905982905982906

==== Trait: Agreeableness ====
Accuracy: 0.1875
F1-score (macro): 0.135632183908046

==== Trait: Emotional stability ====
Accuracy: 0.375
F1-score (macro): 0.33745654458727464


### Random Forest

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns]

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

reports = []

for trait in traits:
    print(f"\n==== Trait: {trait.capitalize()} ====")

    X_train = train_df[feature_cols].values
    y_train = train_df[trait].values

    X_test = val_df[feature_cols].values
    y_test = val_df[trait].values

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train_scaled, y_train)

    y_pred = clf.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print("Accuracy:", acc)
    print("F1-score (macro):", f1)

    reports.append((trait, acc, f1, report))

report_path = "reports/random_forest.txt"
with open(report_path, "w") as f:
    for trait, acc, f1, rep in reports:
        f.write(f"=== Trait: {trait.capitalize()} ===\n")
        f.write(f"Accuracy: {acc:.4f}\n")
        f.write(f"F1-score (macro): {f1:.4f}\n")
        f.write(rep + "\n\n")


==== Trait: Openness ====
Accuracy: 0.625
F1-score (macro): 0.25641025641025644

==== Trait: Conscientiousness ====
Accuracy: 0.65625
F1-score (macro): 0.44805194805194803

==== Trait: Extraversion ====
Accuracy: 0.40625
F1-score (macro): 0.35555555555555557

==== Trait: Agreeableness ====
Accuracy: 0.1875
F1-score (macro): 0.20915032679738563

==== Trait: Emotional stability ====
Accuracy: 0.3125
F1-score (macro): 0.2924867724867725


## Gradient Boosting

In [ ]:
"""
Gradient Boosting Script for predicting the traits separately 
based on the combined CLS-BERT-embeddings + LIWC-features
Evaluates on validation set.
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns]  # Ensure feature overlap

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_test = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

reports = ""

for trait in traits:
    print(f"\n========== Trait: {trait} ==========")

    y_train = train_df[trait].values
    y_test = val_df[trait].values

    clf = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        random_state=42
    )
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    reports += f"\n=== Trait: {trait} ===\n"
    reports += f"Accuracy: {acc:.4f}\n"
    reports += f"F1-score (macro): {f1:.4f}\n"
    reports += report + "\n"

output_path = "reports/gradient_boosting.txt"
with open(output_path, "w") as f:
    f.write("Classification Reports - Gradient Boosting (Train/Val Split)\n")
    f.write(reports)

## MLP classifier

In [ ]:
"""
MLP Classifier Script for predicting the traits separately 
based on the combined CLS-BERT-embeddings + LIWC-features
Evaluates on validation set.
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns] 

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_test = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

reports = ""

for trait in traits:
    print(f"\n========== Trait: {trait} ==========")

    y_train = train_df[trait].values
    y_test = val_df[trait].values

    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    reports += f"\n=== Trait: {trait} ===\n"
    reports += f"Accuracy: {acc:.4f}\n"
    reports += f"F1-score (macro): {f1:.4f}\n"
    reports += report + "\n"

output_path = "reports/mlp.txt"
with open(output_path, "w") as f:
    f.write("Classification Reports - MLP Classifier (Train/Val Split)\n")
    f.write(reports)

## voting ensemble RF, GB, MLP

In [ ]:
"""
Voting Ensemble Classifier (Random Forest + Gradient Boosting + MLP)
Train on train set and evaluate on validation set
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns]  

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_test = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight="balanced")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

ensemble = VotingClassifier(
    estimators=[('rf', rf), ('gb', gb), ('mlp', mlp)],
    voting='soft',
    n_jobs=-1
)

all_reports = ""

for trait in traits:
    print(f"\n========== Predicting {trait.capitalize()} ==========")

    y_train = train_df[trait].values
    y_test = val_df[trait].values

    ensemble.fit(X_train_scaled, y_train)
    y_pred = ensemble.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    all_reports += f"\n\n=== {trait.upper()} ===\n"
    all_reports += f"Accuracy: {acc:.4f}\nF1-score (macro): {f1:.4f}\n"
    all_reports += report

report_path = "reports/rf_gb_mlp.txt"
with open(report_path, "w") as f:
    f.write("Classification Reports - Voting Ensemble (Train/Val Split)\n")
    f.write(all_reports)

In [ ]:
"""
Voting Ensemble Classifier (Random Forest + Gradient Boosting + MLP + SVM)
Train on train set and evaluate on validation set
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns] 

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_test = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight="balanced")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)  
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42) 
svm = SVC(kernel="linear", C=1.0, probability=True, random_state=42, class_weight="balanced")

ensemble = VotingClassifier(
    estimators=[('rf', rf), ('gb', gb), ('mlp', mlp), ('svm', svm)],
    voting='soft',
    n_jobs=-1
)

all_reports = ""

for trait in traits:
    print(f"\n========== Predicting {trait.capitalize()} ==========")

    y_train = train_df[trait].values
    y_test = val_df[trait].values

    ensemble.fit(X_train_scaled, y_train)
    y_pred = ensemble.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    all_reports += f"\n\n=== {trait.upper()} ===\n"
    all_reports += f"Accuracy: {acc:.4f}\nF1-score (macro): {f1:.4f}\n"
    all_reports += report

report_path = "reports/rf_gb_mlp_svm.txt"
with open(report_path, "w") as f:
    f.write("Classification Reports - Voting Ensemble (Train/Val Split)\n")
    f.write(all_reports)

### Ensembles with SMOTE

In [ ]:
"""
Voting Ensemble Classifier (Random Forest + Gradient Boosting + MLP + SVM)
with SMOTE applied separately for each personality trait.
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns] 

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_test = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight="balanced")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
svm = SVC(kernel="linear", C=1.0, probability=True, random_state=42, class_weight="balanced")

ensemble = VotingClassifier(
    estimators=[('rf', rf), ('gb', gb), ('mlp', mlp), ('svm', svm)],
    voting='soft',
    n_jobs=-1
)

all_reports = ""

for trait in traits:
    print(f"\n========== Predicting {trait.capitalize()} ==========")

    y_train = train_df[trait].values
    y_test = val_df[trait].values

    # SMOTE 
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

    ensemble.fit(X_train_balanced, y_train_balanced)
    y_pred = ensemble.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="macro")
    report = classification_report(y_test, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    all_reports += f"\n\n=== {trait.upper()} ===\n"
    all_reports += f"Accuracy: {acc:.4f}\nF1-score (macro): {f1:.4f}\n"
    all_reports += report

report_path = "reports/voting_ensemble_all_traits_val_report.txt"
with open(report_path, "w") as f:
    f.write("Classification Reports - Voting Ensemble with SMOTE (Train/Val Split)\n")
    f.write(all_reports)

In [ ]:
"""
Logistic Regression Script for predicting personality traits
based on CLS-BERT embeddings + LIWC features.
Evaluates on validation set and saves full report.
"""

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

train_df = pd.read_csv("../processed_data/train/comb_train_liwc_embed.csv")
val_df = pd.read_csv("../processed_data/validation/comb_val_liwc_embed.csv")

feature_cols = [col for col in train_df.columns if col.startswith("embed_") or col.startswith("liwc_")]
feature_cols = [col for col in feature_cols if col in val_df.columns]

traits = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Emotional stability"]

scaler = StandardScaler()
X_train = train_df[feature_cols].values
X_val = val_df[feature_cols].values
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

reports = ""

for trait in traits:
    print(f"\n========== Trait: {trait} ==========")
    
    y_train = train_df[trait].values
    y_val = val_df[trait].values

    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_val_scaled)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average="macro")
    report = classification_report(y_val, y_pred, zero_division=0)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score (macro): {f1:.4f}")

    reports += f"\n=== Trait: {trait} ===\n"
    reports += f"Accuracy: {acc:.4f}\n"
    reports += f"F1-score (macro): {f1:.4f}\n"
    reports += report + "\n"

output_path = "reports/logistic_regression.txt"
with open(output_path, "w") as f:
    f.write("Classification Reports - Logistic Regression (Train/Val Split)\n")
    f.write(reports)